In [1]:
import pandas as pd
import numpy as np 
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import xgboost as xgb
from skopt.space import Real, Categorical, Integer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from skopt import BayesSearchCV
from sklearn.utils import resample

## Environment

In [25]:
pip list

Package                            Version             Editable project location
---------------------------------- ------------------- -------------------------
absl-py                            1.0.0
aclpub-check                       0.1                 /Users/zhousiyi/ACLPUB
adjustText                         0.7.3
aiohttp                            3.8.4
aiosignal                          1.3.1
alabaster                          0.7.12
alembic                            1.7.7
anaconda-client                    1.7.2
anaconda-navigator                 2.0.3
anaconda-project                   0.9.1
anyio                              4.2.0
appdirs                            1.4.4
applaunchservices                  0.2.1
appnope                            0.1.4
appscript                          1.1.2
argh                               0.26.2
argon2-cffi                        23.1.0
argon2-cffi-bindings               21.2.0
arrow                              1.3.0
asn1crypto        

## Feature engineering

In [2]:
df1 = pd.read_csv('Data.csv')

In [3]:
df1['Address'] = df1['Address'].apply(lambda x : 'NA' if pd.isnull(x) else x)
df1['if_return'] = df1['Visit.No'].apply(lambda x: True if x>1 else False)

In [4]:
cal_session = min(df1[df1['Session']==2].index)
df1['duration_history2'] = pd.NA
for i in range(cal_session,len(df1)):#The patient's past visits were calculated from the second session
    session = df1.loc[i,'Session']
    if df1.loc[i,'if_return']:#If s/he is a return patient
        id = df1.loc[i,'ID']
        df_temp = df1[(df1['ID']==id)&(df1['Session']<session)]['ServTime'].values#Excluding the data of the same session, only the data of previous sessions can be used for calculation
        if len(df_temp)!=0:
            df1.loc[i,'duration_history2'] = np.mean(df_temp)
        else:
            df1.loc[i,'duration_history2'] = pd.NA#Leave the missing pieces intact and fill in the next 15 minutes
    else:#If s/he is a new patient
        df_temp = df1[(df1['if_return']==False)&(df1['Session']<session)]['ServTime'].values
        df1.loc[i,'duration_history2'] = np.mean(df_temp)

In [5]:
df1['duration_history2'] = df1['duration_history2'].fillna(900)#Take 15 minutes to fill in the missing values

In [6]:
train_start = min(df1[df1['Session']==2].index)
train_end = max(df1[df1['Session']==194].index)
session1_len = len(df1[df1['Session']==1])
def data_process(df,feature,duration_history,y):
    data = df[feature]
    data = pd.get_dummies(data,columns = ['Gender','Address'], drop_first=True)#one-hot
    data_x = data.drop([y],axis = 1)
    data_y = data[y]
    x_train = data_x.loc[:train_end,].copy()
    x_test = data_x.loc[train_end+1:,].copy()
    y_train = data_y.loc[:train_end,].copy()
    y_test = data_y.loc[train_end+1:,].copy()
    for i in [duration_history,'Visit.No']:#normalization
        x1 = np.array(x_train.loc[:,i]).reshape(-1,1)
        scaler = StandardScaler()
        scaler.fit(x1)
        x_train[i+'_scaled'] = scaler.transform(x1).reshape(1,-1)[0]
        x2 = np.array(x_test.loc[:,i]).reshape(-1,1)
        x_test[i+'_scaled'] = scaler.transform(x2).reshape(1,-1)[0]
    x_train = x_train.drop(['Visit.No',duration_history],axis = 1)
    x_test = x_test.drop(['Visit.No',duration_history],axis = 1)
    return x_train, x_test, y_train, y_test

In [7]:
input_feature = ['Visit.No','M.Cancer','S.Cancer','Gender','ServTime','if_return','duration_history2','Address']
x_train, x_test, y_train, y_test = data_process(df1,input_feature,'duration_history2','ServTime')

## Training model


In [8]:
def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [9]:
def eval_model(xtrain_pred,ytrain,predictions,groundtrue):
    print('--------------train----------------')
    print('RMSE',np.sqrt(mean_squared_error(ytrain,xtrain_pred)))
    print('MAPE',MAPE(ytrain,xtrain_pred))
    print('MAE',mean_absolute_error(ytrain,xtrain_pred))
    print('R^2 test: %.3f' % (r2_score(ytrain,xtrain_pred)))
    print('--------------test----------------')
    print('RMSE',np.sqrt(mean_squared_error(groundtrue,predictions)))
    print('MAPE',MAPE(groundtrue,predictions))
    print('MAE',mean_absolute_error(groundtrue,predictions))
    print('R^2 test: %.3f' % (r2_score(groundtrue,predictions)))
    print('--------------eval by group--------------')
    eval = pd.DataFrame({'pred':predictions,'true':groundtrue})
    eval['class'] = eval['pred'].apply(lambda x: 'short' if x<=630.5 else ('long' if x>975.5 else 'median'))
    eval['class2'] = eval['pred'].apply(lambda x: 'short' if x<=811.5 else 'long')
    df = eval.groupby('class').agg({'true':['count',lambda x: x.count() / len(eval),'mean','median','var']})
    print(df)
    df1 = eval.groupby('class2').agg({'true':['count',lambda x: x.count() / len(eval),'mean','median','var']})
    print(df1)

In [10]:
def model_fit(model,params,train_x,train_y,test_x,seed):
    bayes_search = BayesSearchCV(model, params, scoring='neg_mean_squared_error',verbose=2,cv=10,random_state=seed)
    bayes_search.fit(train_x,train_y)
    bestparam = bayes_search.best_params_
    print('bestparam:',bestparam)
    bestmodel = bayes_search.best_estimator_
    pred_test = bestmodel.predict(test_x)
    pred_train = bestmodel.predict(train_x)
    return bayes_search,bestmodel,bestparam,pred_test,pred_train
param_space_xgb = {
    'colsample_bytree': Real(0.4,0.9),
    'colsample_bylevel':Real(0.4,0.9),
    'learning_rate': Categorical([0.1]),
    'lambda':Real(0,200),
    'alpha':Real(0,200),
    'gamma':Real(0,200),
    'n_estimators': Categorical([100,300,500]),
    'max_depth': Integer(2,6),
    'min_child_weight':Integer(1,10)
}

### xgb

In [11]:
xgb_reg = xgb.XGBRegressor(random_state=42)
xgb_bayse,xgb_model,xgb_model_param,xgb_predictions,xgb_train = model_fit(xgb_reg,param_space_xgb,x_train, y_train, x_test, seed = 55)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV] END alpha=158.807024300404, colsample_bylevel=0.7800647945904826, colsample_bytree=0.8501681628173743, gamma=15.910167904632159, lambda=54.557374029227525, learning_rate=0.1, max_depth=3, min_child_weight=4, n_estimators=300; total time=   0.4s
[CV] END alpha=158.807024300404, colsample_bylevel=0.7800647945904826, colsample_bytree=0.8501681628173743, gamma=15.910167904632159, lambda=54.557374029227525, learning_rate=0.1, max_depth=3, min_child_weight=4, n_estimators=300; total time=   0.2s
[CV] END alpha=158.807024300404, colsample_bylevel=0.7800647945904826, colsample_bytree=0.8501681628173743, gamma=15.910167904632159, lambda=54.557374029227525, learning_rate=0.1, max_depth=3, min_child_weight=4, n_estimators=300; total time=   0.3s
[CV] END alpha=158.807024300404, colsample_bylevel=0.7800647945904826, colsample_bytree=0.8501681628173743, gamma=15.910167904632159, lambda=54.557374029227525, learning_rate=0.1, max_depth

In [12]:
eval_model(xgb_train,y_train,xgb_predictions,y_test)

--------------train----------------
RMSE 326.42679730513106
MAPE 34.89118219779518
MAE 240.93698417109587
R^2 test: 0.187
--------------test----------------
RMSE 366.6359920470354
MAPE 40.41809229207963
MAE 268.78459992503605
R^2 test: 0.085
--------------eval by group--------------
        true                                               
       count <lambda_0>         mean  median            var
class                                                      
long     193   0.058168  1050.176166  1003.0  220534.541721
median  2698   0.813140   810.690141   739.0  142286.123455
short    427   0.128692   609.524590   559.0   78939.353267
        true                                             
       count <lambda_0>        mean median            var
class2                                                   
long    1665   0.501808  900.890691  820.0  179974.159919
short   1653   0.498192  695.831821  633.0   92757.020125


In [13]:
test_data = np.hstack([x_test, np.array(y_test).reshape(-1,1)])

# Bootstrap 
n_bootstraps = 1000

# Save the model performance indicators obtained from each resampling
score_rmse = []
score_r_square = []
score_mape = []
score_mae = []

#  Bootstrap 
for i in range(n_bootstraps):
    # Take a number of samples from the test set with put backs to form a new dataset
    test_data_resampled = resample(test_data, replace=True, n_samples=len(test_data), random_state=i)
    x_test_resampled = test_data_resampled[:, :-1]
    y_test_resampled = test_data_resampled[:, -1]

    # Model Evaluation on Bootstrap Resampled Test Sets
    y_pred = xgb_model.predict(x_test_resampled)
    rmse = np.sqrt(mean_squared_error(y_test_resampled, y_pred))
    r2 = r2_score(y_test_resampled,y_pred)
    mape = MAPE(y_test_resampled,y_pred)
    mae = mean_absolute_error(y_test_resampled,y_pred)
    score_rmse.append(rmse)
    score_r_square.append(r2)
    score_mape.append(mape)
    score_mae.append(mae)

# Calculate confidence intervals and means for model performance metrics after Bootstrap resampling
confidence_interval1 = np.percentile(score_rmse, [2.5, 97.5])
mean_score1 = np.mean(score_rmse)
confidence_interval2 = np.percentile(score_r_square, [2.5, 97.5])
mean_score2 = np.mean(score_r_square)
confidence_interval3 = np.percentile(score_mape, [2.5, 97.5])
mean_score3 = np.mean(score_mape)
confidence_interval4 = np.percentile(score_mae,[2.5, 97.5])
mean_score4 = np.mean(score_mae)
print('rmse')
print("Mean performance metrics after Bootstrap resampling: {:.3f}".format(mean_score1))
print("Confidence interval of performance metrics after Bootstrap resampling: [{:.3f}, {:.3f}]".format(confidence_interval1[0], confidence_interval1[1]))
print('r2')
print("Mean performance metrics after Bootstrap resampling: {:.3f}".format(mean_score2))
print("Confidence interval of performance metrics after Bootstrap resampling: [{:.3f}, {:.3f}]".format(confidence_interval2[0], confidence_interval2[1]))
print('mape')
print("Mean performance metrics after Bootstrap resampling: {:.3f}".format(mean_score3))
print("Confidence interval of performance metrics after Bootstrap resampling: [{:.3f}, {:.3f}]".format(confidence_interval3[0], confidence_interval3[1]))
print('mae')
print("Mean performance metrics after Bootstrap resampling: {:.3f}".format(mean_score4))
print("Confidence interval of performance metrics after Bootstrap resampling: [{:.3f}, {:.3f}]".format(confidence_interval4[0], confidence_interval4[1]))

rmse
Mean performance metrics after Bootstrap resampling: 366.150
Confidence interval of performance metrics after Bootstrap resampling: [349.105, 382.330]
r2
Mean performance metrics after Bootstrap resampling: 0.085
Confidence interval of performance metrics after Bootstrap resampling: [0.059, 0.111]
mape
Mean performance metrics after Bootstrap resampling: 40.441
Confidence interval of performance metrics after Bootstrap resampling: [38.949, 41.972]
mae
Mean performance metrics after Bootstrap resampling: 268.646
Confidence interval of performance metrics after Bootstrap resampling: [260.193, 276.885]


### svr

In [14]:
param_space_svr = {
    'C': Real(0.01,100),
    'gamma':Real(0.01,100),
    'kernel': Categorical(['rbf'])
}
svr = SVR()
svr_bayse,svr_model,svr_model_param,svr_predictions,svr_train = model_fit(svr,param_space_svr,x_train, y_train, x_test, seed = 55)


Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV] END C=79.40557179898698, gamma=76.0153576222047, kernel=rbf; total time=   0.5s
[CV] END C=79.40557179898698, gamma=76.0153576222047, kernel=rbf; total time=   0.4s
[CV] END C=79.40557179898698, gamma=76.0153576222047, kernel=rbf; total time=   0.4s
[CV] END C=79.40557179898698, gamma=76.0153576222047, kernel=rbf; total time=   0.4s
[CV] END C=79.40557179898698, gamma=76.0153576222047, kernel=rbf; total time=   0.4s
[CV] END C=79.40557179898698, gamma=76.0153576222047, kernel=rbf; total time=   0.4s
[CV] END C=79.40557179898698, gamma=76.0153576222047, kernel=rbf; total time=   0.4s
[CV] END C=79.40557179898698, gamma=76.0153576222047, kernel=rbf; total time=   0.4s
[CV] END C=79.40557179898698, gamma=76.0153576222047, kernel=rbf; total time=   0.4s
[CV] END C=79.40557179898698, gamma=76.0153576222047, kernel=rbf; total time=   0.4s
Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV] END C=43.54358330440522

/Users/zhousiyi/opt/anaconda3/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV] END ....................C=100.0, gamma=0.01, kernel=rbf; total time=   0.4s
[CV] END ....................C=100.0, gamma=0.01, kernel=rbf; total time=   0.4s
[CV] END ....................C=100.0, gamma=0.01, kernel=rbf; total time=   0.4s
[CV] END ....................C=100.0, gamma=0.01, kernel=rbf; total time=   0.4s
[CV] END ....................C=100.0, gamma=0.01, kernel=rbf; total time=   0.4s
[CV] END ....................C=100.0, gamma=0.01, kernel=rbf; total time=   0.4s
[CV] END ....................C=100.0, gamma=0.01, kernel=rbf; total time=   0.4s
[CV] END ....................C=100.0, gamma=0.01, kernel=rbf; total time=   0.4s
[CV] END ....................C=100.0, gamma=0.01, kernel=rbf; total time=   0.4s
[CV] END ....................C=100.0, gamma=0.01, kernel=rbf; total time=   0.4s
Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV] END C=78.39069152191642, gamma=12.319651313561161, kernel=rbf; 

In [15]:
eval_model(svr_train,y_train,svr_predictions,y_test)

--------------train----------------
RMSE 331.78024557872106
MAPE 30.422580349163603
MAE 228.0508738344116
R^2 test: 0.160
--------------test----------------
RMSE 374.2866873417175
MAPE 37.424124374385734
MAE 266.97405350676826
R^2 test: 0.047
--------------eval by group--------------
        true                                               
       count <lambda_0>         mean  median            var
class                                                      
long      73   0.022001  1219.027397  1278.0  207585.943683
median  2557   0.770645   821.620258   750.0  147170.978979
short    688   0.207354   669.071221   590.0  102873.702344
        true                                             
       count <lambda_0>        mean median            var
class2                                                   
long    1086   0.327306  903.795580  829.5  174355.405179
short   2232   0.672694  747.612455  676.5  125770.934905


In [23]:
# Save the model performance indicators obtained from each resampling
score_rmse = []
score_r_square = []
score_mape = []
score_mae = []

#  Bootstrap 
for i in range(n_bootstraps):
    # Take a number of samples from the test set with put backs to form a new dataset
    test_data_resampled = resample(test_data, replace=True, n_samples=len(test_data), random_state=i)
    x_test_resampled = pd.DataFrame(test_data_resampled[:, :-1],columns=x_train.columns)
    y_test_resampled = test_data_resampled[:, -1]

    # Model Evaluation on Bootstrap Resampled Test Sets
    y_pred = svr_model.predict(x_test_resampled)
    rmse = np.sqrt(mean_squared_error(y_test_resampled, y_pred))
    r2 = r2_score(y_test_resampled,y_pred)
    mape = MAPE(y_test_resampled,y_pred)
    mae = mean_absolute_error(y_test_resampled,y_pred)
    score_rmse.append(rmse)
    score_r_square.append(r2)
    score_mape.append(mape)
    score_mae.append(mae)

# Calculate confidence intervals and means for model performance metrics after Bootstrap resampling
confidence_interval1 = np.percentile(score_rmse, [2.5, 97.5])
mean_score1 = np.mean(score_rmse)
confidence_interval2 = np.percentile(score_r_square, [2.5, 97.5])
mean_score2 = np.mean(score_r_square)
confidence_interval3 = np.percentile(score_mape, [2.5, 97.5])
mean_score3 = np.mean(score_mape)
confidence_interval4 = np.percentile(score_mae,[2.5, 97.5])
mean_score4 = np.mean(score_mae)
print('rmse')
print("Mean performance metrics after Bootstrap resampling: {:.3f}".format(mean_score1))
print("Confidence interval of performance metrics after Bootstrap resampling: [{:.3f}, {:.3f}]".format(confidence_interval1[0], confidence_interval1[1]))
print('r2')
print("Mean performance metrics after Bootstrap resampling: {:.3f}".format(mean_score2))
print("Confidence interval of performance metrics after Bootstrap resampling: [{:.3f}, {:.3f}]".format(confidence_interval2[0], confidence_interval2[1]))
print('mape')
print("Mean performance metrics after Bootstrap resampling: {:.3f}".format(mean_score3))
print("Confidence interval of performance metrics after Bootstrap resampling: [{:.3f}, {:.3f}]".format(confidence_interval3[0], confidence_interval3[1]))
print('mae')
print("Mean performance metrics after Bootstrap resampling: {:.3f}".format(mean_score4))
print("Confidence interval of performance metrics after Bootstrap resampling: [{:.3f}, {:.3f}]".format(confidence_interval4[0], confidence_interval4[1]))

rmse
Mean performance metrics after Bootstrap resampling: 373.665
Confidence interval of performance metrics after Bootstrap resampling: [356.624, 390.968]
r2
Mean performance metrics after Bootstrap resampling: 0.047
Confidence interval of performance metrics after Bootstrap resampling: [0.024, 0.071]
mape
Mean performance metrics after Bootstrap resampling: 37.436
Confidence interval of performance metrics after Bootstrap resampling: [36.086, 38.794]
mae
Mean performance metrics after Bootstrap resampling: 266.763
Confidence interval of performance metrics after Bootstrap resampling: [257.939, 275.425]


### lr

In [22]:
lr = LinearRegression()
lr.fit(x_train,y_train)
print('coef',lr.coef_)
print('intercept',lr.intercept_)

lr_predictions = lr.predict(x_test)
lr_train = lr.predict(x_train)
eval_model(lr_train,y_train,lr_predictions,y_test)


coef [ 157.15177678  115.43417208 -187.4018734   -13.77195628   14.40759647
   46.69267981  163.23042705   52.88481447   11.17336462]
intercept 887.4299963720251
--------------train----------------
RMSE 340.8302635222
MAPE 36.838510800963164
MAE 252.81531198287507
R^2 test: 0.114
--------------test----------------
RMSE 367.15745478915426
MAPE 41.45027744171662
MAE 272.9112891798674
R^2 test: 0.083
--------------eval by group--------------
        true                                              
       count <lambda_0>         mean median            var
class                                                     
long     255   0.076854  1029.101961  963.0  200058.170665
median  2874   0.866184   792.733820  720.0  138756.654846
short    189   0.056962   579.126984  492.0   81793.324215
        true                                             
       count <lambda_0>        mean median            var
class2                                                   
long    1743   0.525316  884.

In [24]:
# Save the model performance indicators obtained from each resampling
score_rmse = []
score_r_square = []
score_mape = []
score_mae = []

#  Bootstrap 
for i in range(n_bootstraps):
    # Take a number of samples from the test set with put backs to form a new dataset
    test_data_resampled = resample(test_data, replace=True, n_samples=len(test_data), random_state=i)
    x_test_resampled = pd.DataFrame(test_data_resampled[:, :-1],columns=x_train.columns)
    y_test_resampled = test_data_resampled[:, -1]

    # Model Evaluation on Bootstrap Resampled Test Sets
    y_pred = lr.predict(x_test_resampled)
    rmse = np.sqrt(mean_squared_error(y_test_resampled, y_pred))
    r2 = r2_score(y_test_resampled,y_pred)
    mape = MAPE(y_test_resampled,y_pred)
    mae = mean_absolute_error(y_test_resampled,y_pred)
    score_rmse.append(rmse)
    score_r_square.append(r2)
    score_mape.append(mape)
    score_mae.append(mae)

# Calculate confidence intervals and means for model performance metrics after Bootstrap resampling
confidence_interval1 = np.percentile(score_rmse, [2.5, 97.5])
mean_score1 = np.mean(score_rmse)
confidence_interval2 = np.percentile(score_r_square, [2.5, 97.5])
mean_score2 = np.mean(score_r_square)
confidence_interval3 = np.percentile(score_mape, [2.5, 97.5])
mean_score3 = np.mean(score_mape)
confidence_interval4 = np.percentile(score_mae,[2.5, 97.5])
mean_score4 = np.mean(score_mae)
print('rmse')
print("Mean performance metrics after Bootstrap resampling: {:.3f}".format(mean_score1))
print("Confidence interval of performance metrics after Bootstrap resampling: [{:.3f}, {:.3f}]".format(confidence_interval1[0], confidence_interval1[1]))
print('r2')
print("Mean performance metrics after Bootstrap resampling: {:.3f}".format(mean_score2))
print("Confidence interval of performance metrics after Bootstrap resampling: [{:.3f}, {:.3f}]".format(confidence_interval2[0], confidence_interval2[1]))
print('mape')
print("Mean performance metrics after Bootstrap resampling: {:.3f}".format(mean_score3))
print("Confidence interval of performance metrics after Bootstrap resampling: [{:.3f}, {:.3f}]".format(confidence_interval3[0], confidence_interval3[1]))
print('mae')
print("Mean performance metrics after Bootstrap resampling: {:.3f}".format(mean_score4))
print("Confidence interval of performance metrics after Bootstrap resampling: [{:.3f}, {:.3f}]".format(confidence_interval4[0], confidence_interval4[1]))

rmse
Mean performance metrics after Bootstrap resampling: 366.654
Confidence interval of performance metrics after Bootstrap resampling: [351.120, 382.334]
r2
Mean performance metrics after Bootstrap resampling: 0.082
Confidence interval of performance metrics after Bootstrap resampling: [0.062, 0.102]
mape
Mean performance metrics after Bootstrap resampling: 41.466
Confidence interval of performance metrics after Bootstrap resampling: [39.979, 42.992]
mae
Mean performance metrics after Bootstrap resampling: 272.749
Confidence interval of performance metrics after Bootstrap resampling: [264.342, 280.785]
